<!-- @format -->

#### Do all the imports


In [1]:
import requests
from dotenv import load_dotenv
from os import getenv
from json import dump

<!-- @format -->

#### load the environments


In [2]:
load_dotenv("techtalk_SEC.secrets")
load_dotenv("techtalk_SEC.env")

True

<!-- @format -->

#### prepare the request


In [3]:
r = requests.Request(
    method="GET",
    url=f'{getenv("WEB_URL")}/files/company_tickers.json',
    headers={
        "User-Agent": "PWTTechTalkBot/1.0 Port Wallis Technologies contact - rwolfe@portwallistechnologies.com",
        "Accept": "application/json",
    },
)

s = requests.Session()

<!-- @format -->

#### Go get the data


In [4]:
req = s.send(r.prepare())

<!-- @format -->

#### Save the response to an external file (completely optional)


In [5]:
with open(file="cik_tickers.json", mode="w") as file:
    dump(req.json(), file)

#### stash the ticker mapping into a dict for later


In [10]:
the_json = req.json()
ticker_cik = {the_json[a]["ticker"]: str(the_json[a]["cik_str"]).zfill(10) for a in the_json}
cik_ticker = {str(the_json[a]["cik_str"]).zfill(10): the_json[a]["ticker"] for a in the_json}

<!-- @format -->

#### Pull your favorite company's CIK from the dictionary 

In [11]:
ticker_symbol = "AES"

company_cik = ticker_cik[ticker_symbol]

<!-- @format -->

#### And go get the list of recent submissions for that company


In [12]:
r.url = f"""{getenv("DATA_URL")}/submissions/CIK{company_cik}.json"""

req = s.send(r.prepare())

<!-- @format -->

#### Save the results to a file again


In [13]:
with open(file=f"submissions-{ticker_symbol}.json", mode="w") as file:
    dump(req.json(), file)

<!-- @format -->

#### save the submissions to a couple of dicts for later use


In [14]:
submissions: dict = req.json()

recent_filings: dict = submissions["filings"]["recent"]

submission_count = len(recent_filings["accessionNumber"])

sub_list = []

for i in range(submission_count):
    sub_list.append({k: recent_filings[k][i] for k in recent_filings.keys()})

In [15]:
form_names = set()

toss_me = [
    form_names.add(sub_list[i]["form"])
    for i in range(len(sub_list))
    if sub_list[i]["form"] not in form_names
]

all_forms: dict[str:list] = {k: list() for k in form_names}


toss_me = [
    all_forms[sub_list[i]["form"]].append(sub_list[i]) for i in range(submission_count)
]

del toss_me